#### 네이버 뉴스 sub 페이지 크롤링
    - 한개의 sub 페이지 크롤링 한 후 함수로 재구성
    - 02_1번에서 수집된 데이터를 이용해서 여러 subpage 크롤링
    


In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen




In [10]:

# 뉴스 sub 페이지

headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004833477"
result = requests.get(url, headers=headers)


In [11]:
result


<Response [200]>

In [12]:
#result.content # 전송된소스코드를 반환 - row text 형식으로 반환 앞에 b 코드가 붙어있음
# result.text # 전송된 소스코드를 문자열로 반환

In [13]:
bs_obj = BeautifulSoup(result.content, "html.parser")
## 기사 제목 추출
title = bs_obj.find(id='articleTitle')
# title.text

## 기사 입력 시간
date_time = bs_obj.find("span",{"class":"t11"})
# date_time.text

## 좋아요 수 (동적으로 실시간으로 변해서 다른방법을 사용해야함 셀레니움 패키지 사용)
like_count = bs_obj.find("span",{"class":"u_likeit_text _count num"})
# bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")
like_count


#### 개발자 도구의 copy selector 사용방버

- 크롤링하려는 컨텐츠에서 검사 - 해당 소스에서 단축메뉴 - copy - copy selector
    - 붙여넣기 하면 선택자를 반환해주는데 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 - 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적 크롤링 불가능
            - 네이버 뉴스 서브페이지의 좋아요 수와 기사 댓글 수는 동적 생성이되어서 정적 크롤링 불가능
            - 셀레니움이라는 패키지를 사용해야 함
            
bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")

In [14]:
# sub page 에서 수집된 데이터 반환하는  함수 작성

def get_news_sub_info(url) :

    # 1. 전달된 url에 접속 후 응답 객체 반환
    headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요데이터 추출
    ## 기사 제목 추출 (태그객체)
    title = bs_obj.find(id='articleTitle')
    ## 기사 입력 시간 (태그객체)
    date_time = bs_obj.find("span",{"class":"t11"})
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = date_time.text
    
    return dict_sub

In [17]:
## main : 함수 호출 

headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com"
res = requests.get(url, headers=headers)
html = res.text

bs_obj = BeautifulSoup(html, "html.parser")

# 헤드라인 뉴스의 구성 ul , class : hdline_article_list

news_list = bs_obj.find("ul", {"class":"hdline_article_list"})

# news_list


lis = news_list.findAll("li")

link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])

    
# link_list
for url in link_list :
    res = get_news_sub_info(url)
    print(res)

{'기사제목': '"한미훈련, 여러 가지 고려해 신중 협의" 文 언급 뜻?', '기사 입력 시간': '2021.08.05. 오전 7:32'}
{'기사제목': '‘톱4’ 빠진 국힘 대선주자 봉사활동… 모양새 구긴 이준석', '기사 입력 시간': '2021.08.05. 오전 6:06'}
{'기사제목': '‘2050 탄소중립’ 한다더니…‘포기’ 시나리오 검토하는 탄소중립위원회', '기사 입력 시간': '2021.08.05. 오전 10:02'}
{'기사제목': '내년도 최저임금 9,160원…1월 1일부터 적용', '기사 입력 시간': '2021.08.05. 오전 6:08'}
{'기사제목': '전국 찜통 더위… 수도권 곳곳서 소나기 소식도', '기사 입력 시간': '2021.08.05. 오전 6:19'}


#### 연습문제
- get_news_sub_info 함수의 반환겂에 기사 내용까지 포함되게 코드를 수정하고
- 반환되는 결과값을 df로 구성하시오.



In [21]:

def get_news_sub_info(url) :

    # 1. 전달된 url에 접속 후 응답 객체 반환
    headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요데이터 추출
    ## 기사 제목 추출 (태그객체)
    title = bs_obj.find(id='articleTitle')
    ## 기사 입력 시간 (태그객체)
    date_time = bs_obj.find("span",{"class":"t11"})
    
    #기사 내용 추출(id=articleBodyContents)
    contents = bs_obj.find(id = "articleBodyContents")
    
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = date_time.text
    dict_sub["기사"] = contents.text
    return dict_sub

- 함수 끝


### main 시작

In [22]:
# 빈 데이터 프레임 하나 생성
import pandas as pd
dict_sub = {}
dict_sub["기사제목"] = []
dict_sub["기사 입력 시간"] = []
dict_sub["기사"] = []
news_sub = pd.DataFrame(dict_sub)
news_sub

,기사제목,기사 입력 시간,기사


In [27]:
## main : 함수 호출 
 
headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com"
res = requests.get(url, headers=headers)
html = res.text

bs_obj = BeautifulSoup(html, "html.parser")

# 헤드라인 뉴스의 구성 ul , class : hdline_article_list

news_list = bs_obj.find("ul", {"class":"hdline_article_list"})

# news_list


lis = news_list.findAll("li")

link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])

    
# link_list
for url in link_list :
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub = news_sub.append(res, ignore_index=True)

news_sub

,기사제목,기사 입력 시간,기사
0,"“쩍벌 조금 줄어들었나요?” 윤석열, 반려견과 ‘금쩍 운동’",2021.08.05. 오전 9:50,\n\n\n\n\n국민의힘 대권주자인 윤석열 전 검찰총장이 ‘쩍벌’(다리를 넓게 벌...
1,"'부동산 책임론' 포문 열자, '음주운전 전력' 꼬집었다",2021.08.05. 오전 7:22,\n\n\n\n동영상 뉴스\n\n\n\n\n민주당 대선 경선 2차 TV 토론회 장면...
2,올해 수능 방역용 책상 ‘칸막이’ 없이 실시,2021.08.05. 오전 10:03,"\n\n\n\n\n교육부, 올해 11월 수능 때 책상 칸막이 미설치점심식사할 때만 ..."
3,"與, 한미훈련 연기론 확산…범여권 의원 60명 연판장",2021.08.05. 오전 8:10,\n\n\n\n\n사진 제공:연합뉴스더불어민주당 등 범여권 의원 60여명이 이달 예...
4,내년 최저임금 9160원 확정 고시…올해보다 440원↑,2021.08.05. 오전 8:32,"\n\n\n\n\n\t\n\t[사용자 측 이의제기 불수용하고 최종 확정고시, 월급 ..."
5,김밥집 식중독 130명 넘어…팔린 김밥만 4천200여 줄,2021.08.05. 오전 6:19,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶경기도...
6,"“미, 백신 접종완료 외국인만 입국허용 계획 입안 중”",2021.08.05. 오전 9:27,\n\n\n\n\n미국 행정부는 코로나19 백신 접종을 완료한 외국인에 대해서만 미...
7,"'부동산 책임론' 포문 열자, '음주운전 전력' 꼬집었다",2021.08.05. 오전 7:22,\n\n\n\n동영상 뉴스\n\n\n\n\n민주당 대선 경선 2차 TV 토론회 장면...
8,"SK이노베이션, 주가하락 과도…본업 시황 감안해야 -신한",2021.08.05. 오전 7:59,\n\n\n\n\n\t\n\t[이데일리 김소연 기자] 신한금융투자는 5일 SK이노베...
9,"'제품 개발 속도 당긴다'…금호타이어, 빅데이터·AI로 디지털 전환 박차",2021.08.05. 오전 9:11,\n\n\n\n\n금호타이어가 타이어 개발 과정에 빅데이터와 인공지능(AI) 기술을...


In [ ]:
### 기존에 수집한 데이터

In [30]:
news_df = pd.read_csv("./crawl_data/naver_news.csv", index_col = 0)

In [33]:
news_df

,section,news_title,link_url
0,헤드라인,‘성추행 확인’ 쿠오모 뉴욕주지사 사면초가,https://news.naver.com/main/read.naver?mode=LS...
1,헤드라인,"교회서 하루 80여 명 확진… 대구, 1차 대유행 후 최악 확산세",https://news.naver.com/main/read.naver?mode=LS...
2,헤드라인,"‘불법 집회’ 민노총위원장, 한달만에 첫조사",https://news.naver.com/main/read.naver?mode=LS...
3,헤드라인,"""SK이노베이션, 지주사 할인 적용해 투자의견·목표가↓""-하나금투",https://news.naver.com/main/read.naver?mode=LS...
4,헤드라인,"제 2의 게임스톱?…로빈후드 주가, 이틀만에 90% 급등",https://news.naver.com/main/read.naver?mode=LS...
5,정치,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",https://news.naver.com/main/read.naver?mode=LS...
6,정치,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",https://news.naver.com/main/read.naver?mode=LS...
7,정치,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",https://news.naver.com/main/read.naver?mode=LS...
8,정치,"이준석 ""안철수, 그냥 내가 싫은 거 아닌가?""",https://news.naver.com/main/read.naver?mode=LS...
9,정치,"'도리도리 점검단'…윤석열, '쩍벌' 이어 또 셀프디스",https://news.naver.com/main/read.naver?mode=LS...


In [34]:
# 빈 데이터프레임 하나 생성
import pandas as pd
dict_sub={} # 빈 dict
dict_sub["기사제목"] = [] #dict에 원소 생성
dict_sub["기사 입력 시간"]=[]
dict_sub["기사"] = []
news_sub = pd.DataFrame(dict_sub) #df를 생성
news_sub


,기사제목,기사 입력 시간,기사


In [39]:
import time

link_list = news_df["link_url"]

for url in link_list :
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub_main = news_sub_main.append(res, ignore_index=True)
    time.sleep(2) # 계속되는 요청으로 서버가 disconnect 시킬 수 있으므로 시간 간격을 둔다.

news_sub_main

,기사제목,기사 입력 시간,기사
0,"내년 최저임금, 올해보다 5.1% 오른 9,160원 확정",2021.08.05. 오전 7:58,"\n\n\n\n\n고용부, 5일 고시 관보 게재근로자 최대 355만명 영향경영계, ..."
1,또 '음주운전·탄핵' 과거사 생채기…검증단 동의했지만 시각차 여전,2021.08.05. 오전 5:01,\n\n\n\n\n핵심요약민주 대선 경선 2차 TV토론회'음주운전·백제' 등 선두 ...
2,靑 장·차관급 인사…인권위원장 송두환·금융위원장 고승범,2021.08.05. 오전 10:34,"\n\n\n\n\n장관급 2명·차관급 6명 인사인권위원장에 송두환(왼쪽), 금융위원..."
3,"김밥 먹고 134명 식중독…""4,200줄 팔았는데""",2021.08.04. 오후 11:10,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t<앵커>경기도 성남 분당에 있...
4,[속보] ‘신천지 집단감염’ 이후 최다…대구 121명 확진,2021.08.05. 오전 8:25,"\n\n\n\n\n교회, 태권도장 중심으로 확진자 급증대구지역 코로나19 확진자가 ..."
...,...,...,...
71,"미스터리한 곰팡이, 우주로 떠난다 [우주로 간다]",2021.08.05. 오전 9:14,\n\n\n\n\n\t\n\t'블롭’(Blob)이라고 불리는 미스터리한 단세포 생명...
72,"배터리 3사, 2분기 반도체 부족 불구 선방…3분기 '전원 흑자' 기대",2021.08.05. 오전 9:14,"\n\n\n\n\n - LG에너지솔루션, IPO 앞두고 ‘SK 보상금’ 포함 -..."
73,"삼성, 갤럭시 언팩 기대감↑…트레일러 영상 조회수 1억 돌파",2021.08.05. 오전 9:07,\n\n\n\n\n삼성전자가 지난달 28일 공개한 '갤럭시 언팩'(공개) 행사 공식...
74,"""폴더블 관심 UP""…'삼성 갤럭시 언팩' 트레일러 영상 1억뷰 돌파",2021.08.05. 오전 9:04,\n\n\n\n\n시간당 평균 57만명 시청…역대 언팩 트레일러 중 가장 빠른 속도...
